### 4) Поэкспериментировать с ALS (grid-search)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from sklearn.model_selection import GridSearchCV

from metrics import precision_at_k, recall_at_k

In [3]:
data = pd.read_csv('retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(5)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [6]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [7]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Progs\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [9]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [11]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [12]:
%%time

params = {'factors':[ 40, 80, 120, 140], 
          'regularization':[ 0.001, 0.05]}

for factor in params['factors']:
    for reg in params['regularization']:
        model = AlternatingLeastSquares(factors=factor, 
                                regularization=reg,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4,
                                random_state=42)

        model.fit(csr_matrix(user_item_matrix).T,  # На вход item-user matrix
                  show_progress=False)


        result['als_bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
        sc = result.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean()
        print(f'Factor:{factor:3},  Regularization:{reg:6}: p_at_k:{sc:7.5f}')

Factor: 40,  Regularization: 0.001: p_at_k:0.16562
Factor: 40,  Regularization:  0.05: p_at_k:0.16856
Factor: 80,  Regularization: 0.001: p_at_k:0.18541
Factor: 80,  Regularization:  0.05: p_at_k:0.18599
Factor:120,  Regularization: 0.001: p_at_k:0.19432
Factor:120,  Regularization:  0.05: p_at_k:0.19491
Factor:140,  Regularization: 0.001: p_at_k:0.20069
Factor:140,  Regularization:  0.05: p_at_k:0.19902
Wall time: 5min 23s


In [13]:
params = {'factors':[ 160, 180, 200], 
          'regularization':[ 0.001, 0.05]}

for factor in params['factors']:
    for reg in params['regularization']:
        model = AlternatingLeastSquares(factors=factor, 
                                regularization=reg,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4,
                                random_state=42)

        model.fit(csr_matrix(user_item_matrix).T,  # На вход item-user matrix
                  show_progress=False)


        result['als_bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
        sc = result.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean()
        print(f'Factor:{factor:3},  Regularization:{reg:6}: p_at_k:{sc:7.5f}')

Factor:160,  Regularization: 0.001: p_at_k:0.19971
Factor:160,  Regularization:  0.05: p_at_k:0.20020
Factor:180,  Regularization: 0.001: p_at_k:0.19726
Factor:180,  Regularization:  0.05: p_at_k:0.19814
Factor:200,  Regularization: 0.001: p_at_k:0.19824
Factor:200,  Regularization:  0.05: p_at_k:0.19638


Оптимальными являются параметры: Factor:140,  Regularization: 0.001.